In [1]:
# -*- coding: utf-8 -*-
"""
import_qualtrics_surveys- Uses Qualtrics API to automate the import and activation of the meaning map patch rating surveys.

@author: Taylor R. Hayes
"""

#%% 010: Import modules and define relative folder path

# Set your project ID and your API token here
# MSB : mUYDofbqRqWRh57qOcHr1vBjwTBWpKwT6PATbaHI   
apiToken = "mUYDofbqRqWRh57qOcHr1vBjwTBWpKwT6PATbaHI" # This can be found in your Qualtrics account settings->Qualtrics IDs

import os
import re
import requests
import json
from datetime import datetime  
from datetime import timedelta 
from natsort import natsorted, ns
from time import sleep

relpath = os.path.dirname(os.path.realpath('__file__'))

In [2]:
#%% 020: Define various API functions

###################
# Import Survey API 
###################
def import_survey(survey,survey_file,name,apiToken):    
    
    # Setting user Parameters
    dataCenter = "co1"	# UC Davis ID
    
    # Defining inputs
    baseUrl = "https://ucdavis.co1.qualtrics.com/API/v3/surveys".format(dataCenter)
    headers = {
        "x-api-token": apiToken,
        }
    
    files = {
        'file': (survey, open(survey_file, 'rb'), 'application/vnd.qualtrics.survey.txt')
        }
    
    data = { "name": name }
    
    # Make response and print output
    response = requests.post(baseUrl, files=files, data=data, headers=headers)
    print(response.text)
    return response


###############
# Update Survey
###############

def update_survey(surveyId,name,apiToken):
    
    # Setting user Parameters
    dataCenter = "co1"	
    
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}".format(dataCenter, surveyId)
    headers = {
        "content-type": "application/json",
        "x-api-token": apiToken,
        }
    
    data = { 
        "name": name, 
        "isActive": True, 
        "expiration": { 
            "startDate": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"), 
            "endDate": (datetime.now()+timedelta(days=90)).strftime("%Y-%m-%dT%H:%M:%SZ")
        } 
      }
    
    response = requests.put(baseUrl, json=data, headers=headers)
    print(response.text)


################
# Get Survey API
################

def get_survey(surveyId,apiToken):

    # Setting user Parameters
    dataCenter = "co1"	
    
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}".format(dataCenter, surveyId)
    headers = {
        "x-api-token": apiToken,
        }
    
    response = requests.get(baseUrl, headers=headers)
    #print(response.text)
    return response

###################
# Delete Survey API
###################

def delete_survey(surveyId,apiToken):
    
    # Setting user Parameters
    dataCenter = "co1"	
    
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys/{1}".format(dataCenter, surveyId)
    headers = {
        "x-api-token": apiToken,
        }
    
    response = requests.delete(baseUrl, headers=headers)
    print(response.text)

In [3]:
task_path = relpath

os.path.isdir(task_path)

True

NameError: name 'task' is not defined

In [10]:
all_ids = []

for task in os.listdir(task_path):
    if os.path.isdir(os.path.join(task_path,task)) and task in ['meaning']:
        relpath = os.path.join(task_path,task)

        project_ID = 'EDesc0203_'+task.split('_')[0] # This will be the name stem for each survey

        #%% 030: Import and generate all fine patch jobs in Qualtrics

        #-- Get fine survey files
        fine_path = os.path.join(relpath,'edesc02fine')
        fine_files = os.listdir(fine_path)
        fine_files = natsorted(fine_files, alg=ns.IGNORECASE)

        #-- For each file
        for x in fine_files:

            #-- Create survey
            survey_file = os.path.join(fine_path, x)
            survey_number = re.findall("\d+", x)
            name = project_ID + '_fine' +  survey_number[0].rjust(3,"0") # Qualtrics name sort

            #-- Call API to import survey
            response = import_survey(x, survey_file, name, apiToken)

            #-- Extract survey id from response.text
            response_list = re.findall('"([^"]*)"', response.text)
            surveyId = response_list[2] 

            all_ids.append(surveyId)
            
            #-- Activate survey
            update_survey(surveyId,name,apiToken)
            
            sleep(1)


        #%% 040: Import and generate all coarse patch jobs in Qualtrics

        #-- Get fine survey files
        coarse_path = os.path.join(relpath,'edesc02coarse')
        coarse_files = os.listdir(coarse_path)
        coarse_files = natsorted(coarse_files, alg=ns.IGNORECASE)

        #-- For each file
        for x in coarse_files:

            #-- Create survey
            survey_file = os.path.join(coarse_path, x)
            survey_number = re.findall("\d+", x)
            name = project_ID + '_coarse' +  survey_number[0].rjust(3,"0") # Qualtrics name sort

            #-- Call API to import survey
            response = import_survey(x, survey_file, name, apiToken)

            #-- Extract survey id from response.text
            response_list = re.findall('"([^"]*)"', response.text)
            surveyId = response_list[2] 
            
            all_ids.append(surveyId)

            #-- Activate survey
            update_survey(surveyId,name,apiToken)
            
            sleep(1)

        # END OF IMPORT_QUALTRICS_SURVEYS

with open(os.path.join(task_path,task,'survey_ids.txt'),'w') as outfile:
    for id_str in all_ids:
        outfile.write(id_str+"\n")
outfile.close()

NotADirectoryError: [Errno 20] Not a directory: '/Users/madisonbarker/Documents/GitHub/msbarker3.github.io/Projects/Linearization/EDesc02_Qualtrics_Survey_Upload.ipynb/survey_ids.txt'

In [23]:
with open(os.path.join(task_path,task,'survey_ids.txt'),'w') as outfile:
    for id_str in all_ids:
        outfile.write(id_str+"\n")
outfile.close()

In [5]:
task = 'grasp' # Set to the task surveys to populate

existing_surveys = open(os.path.join(task_path,task,'survey_ids.txt'),'r').readlines()
existing_surveys = [d.replace('\n','') for d in existing_surveys]

In [13]:
# Retrieve surveys from Qualtrics to get actual QIDs

get_survey_path = os.path.join(task_path,task,'from_qualtrics')

if not os.path.isdir(get_survey_path):
    os.mkdir(get_survey_path)

for f in existing_surveys:
    tmp = get_survey(f,apiToken)
    text = tmp.text
    name = text.split('"name":"')[1].split('"')[0]+'.txt'
    f = open(os.path.join(get_survey_path,name),'w',encoding='UTF-8')
    f.write(text)
    f.close()       

In [11]:
os.path.join(get_survey_path,name)

'C:\\Users\\gwenz\\Box\\FerreiraLabShare\\TDrive backup\\Gwen\\MeaningMap_Verbal\\Running\\Stimuli\\SDesc01\\mapping\\surveys\\grasp\\from_qualtrics\\SDesc01_grasp_fine001.txt'

In [9]:
# Delete surveys
task = 'interact' # Set to the task surveys to be deleted
to_delete = existing_surveys

for survey in to_delete:
    delete_survey(survey,apiToken)

{"meta":{"httpStatus":"200 - OK","requestId":"29be6894-43b1-4a13-85e1-5959d1306f33"}}
{"meta":{"httpStatus":"200 - OK","requestId":"f9111bb0-435b-43bb-b31e-06397096035b"}}
{"meta":{"httpStatus":"200 - OK","requestId":"e7a765d5-ab01-4e12-8df5-e7b0f59718cb"}}
{"meta":{"httpStatus":"200 - OK","requestId":"d11d9f20-7f56-4969-a7f2-e5e222395646"}}
{"meta":{"httpStatus":"200 - OK","requestId":"cf13de1f-00bb-4231-b2f6-5c7f11dfef42"}}
{"meta":{"httpStatus":"200 - OK","requestId":"adbd8f75-5725-428f-aa8e-b67fd54ed6de"}}
{"meta":{"httpStatus":"200 - OK","requestId":"30dc9cfe-4f67-431b-a75e-fade452e18f0"}}
{"meta":{"httpStatus":"200 - OK","requestId":"da83cdc5-ce02-4c02-83a1-88ad290c2dc6"}}
{"meta":{"httpStatus":"200 - OK","requestId":"69570b63-ee85-44f4-a300-2ea5d0fc1cb1"}}
{"meta":{"httpStatus":"200 - OK","requestId":"1f6eed02-53ac-48ca-ba35-3fd3503593e1"}}
{"meta":{"httpStatus":"200 - OK","requestId":"d0803ac0-8214-4f99-b827-ca1c120632bf"}}
{"meta":{"httpStatus":"200 - OK","requestId":"5027b9b6